# Types and Dispatch in Julia

One of the most important goals of high-level languages is to provide *polymorphism*: the ability for the same code to operate on different kinds of values.

Julia uses a vocabulary of *types* for this purpose. Types play the following roles:

- Describe "what kind of thing is this"
- Describe the representation of a value
- Driving *dispatch*: selecting one of several pieces of code
- Driving *specialization*: code is optimized by assuming values have certain types

## Describing values

In [ ]:
typeof(3)

In [ ]:
sizeof(Int64)

In [ ]:
Int64.size

In [ ]:
isbits(Int64)

In [ ]:
Int64.mutable

In [ ]:
supertype(Int64)

In [ ]:
supertype(Signed)

In [ ]:
supertype(Integer)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

In [ ]:
# The subtype operator/relation
Integer <: Real

In [ ]:
String <: Real

In [ ]:
Any >: String

In [ ]:
# The `isa` operator/relation
1 isa Int

In [ ]:
1 isa String

Julia has roughly 5 kinds of types. We just saw two:

1. Data types - describing concrete data objects
2. Abstract types - group those together

There are three more:

1. Union types
2. UnionAll types
3. The empty type

## Union types

Expresses a *set union* of types.

In [ ]:
1 isa Union{Int,String}

In [ ]:
"hi" isa Union{Int,String}

## UnionAll types

Expresses an *iterated set union* of types.

In [ ]:
[1] isa Vector{Int}

In [ ]:
[1] isa (Vector{T} where T<:Real)

$\bigcup\limits_{T<:Real} \tt{Vector}\{T\}$

In [ ]:
Union{Vector{Any},Vector{Real}} <: Vector{T} where T>:Real

In [ ]:
T where T<:Real

In [ ]:
rand(1:10,2,2)

In [ ]:
dump(Array)

In [ ]:
Vector

In [ ]:
Vector{Int} <: Vector

In [ ]:
Vector <: Array

In [ ]:
Vector{Int} <: Vector{Any}

In [ ]:
typeintersect((Array{T} where T<:Real), (Array{T,2} where T>:Int))

In [ ]:
[[2]] isa (Vector{T} where T<:Vector{S} where S<:Integer)

## The empty type

Corresponds to the empty set.

In [ ]:
1 isa Union{}

In [ ]:
Union{} <: Int

In [ ]:
Union{} <: String

In [ ]:
Union{} <: Array

This represents situations where there can't be any value; e.g. an exception is thrown or the program doesn't terminate.

## Dispatch

In [ ]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

In [ ]:
methods(f)

In [ ]:
f(1.5, 2)

In [ ]:
f(1, "string")

In [ ]:
f(1, 2)

In [ ]:
f(1, 2, 3)

## Tuples

A tuple is an immutable container of any combination of values.

Often used to represent e.g. ordered pairs, or for returning "multiple" values from functions.

In [ ]:
t = (1, "hi", 0.33, pi)

In [ ]:
t[2]

In [ ]:
# "destructuring"
a, b, c = t

In [ ]:
a

In [ ]:
b

In [ ]:
typeof(t)

Tuple types represent the arguments to a function.

In [ ]:
first(methods(f)).sig

For every function call, the method that gets called is the most specific one such that the argument tuple type is a subtype of the signature.

## "Diagonal" dispatch

In [ ]:
d(x::T, y::T) where {T} = "same type"
d(x, y) = "different types"

In [ ]:
d(1, 1)

In [ ]:
d(1, 2.0)

In [ ]:
[ m.sig for m in methods(d) ]

## Variadic (or varargs) methods

In [ ]:
v(x...) = (x, "zero or more")

In [ ]:
v(x, xs...) = (xs, "one or more")

In [ ]:
v()

In [ ]:
v(1)

In [ ]:
v(1, 2, 3, 4)

## Variadic tuple types

In [ ]:
foo(a::Array, Is::Int...) = 0

In [ ]:
first(methods(foo)).sig

In [ ]:
vt = Tuple{Array, Vararg{Int}}

In [ ]:
isa(([1],1,2,3), vt)

In [ ]:
isa(([1],1,0.02,3), vt)

## Specialization in action

Internally, the compiler generates specializations for particular types.

Example: For a 3-argument function `f`, the compiler might decide to generate a specialization for `Tuple{Int, Any, Int}`, if for some reason the second argument isn't important.

In [ ]:
addall(t) = +(t...)  # "splat"

In [ ]:
@code_typed addall((1,2))

In [ ]:
@code_typed addall((1,2,3))

In [ ]:
function alltrue(f, itr)
    @inbounds for x in itr
        f(x) || return false
    end
    return true
end

In [ ]:
@which isinteger(1)

In [ ]:
@code_typed alltrue(isinteger, [1,2,3])

In [ ]:
@code_llvm alltrue(isinteger, [1,2,3])

## Dispatch, specialization, and performance

Dynamic dispatch is traditionally considered "slow".

Instead of a `call` instruction, you need to do a table lookup procedure first.

However:
1. If types are known, the call target can be looked up at compile time.
2. The cost of dynamic dispatch is well worth it *if* you're dispatching to an optimized kernel.

## What to specialize on?

We can't specialize on *everything* because it would take too long and generate too much code.

There's no fully general and automatic approach.

We specialize on types. That's a reasonable default. If the default's not good enough, move more information into types!

A classic: specializing on the value of an integer.

In [ ]:
function sum1n(::Val{N}) where {N}    # given `struct Val{N} end`
    s = 0
    for i = 1:N
        s += i
    end
    return s
end

In [ ]:
sum1n(Val{10}())

In [ ]:
@code_llvm sum1n(Val{10}())

In [ ]:
sum1n(n::Integer) = sum1n(Val{n}())

In [ ]:
sum1n(20)

In [ ]:
sum1n(rand(1:100))  # dynamic dispatch to specialized code

# "Stupid Dispatch Tricks"

## Trick 1: processing arguments recursively

The compiler's optimizations can be exploited to move parts of your own computations to compile time (thus saving time at run time). The general idea is to represent more information within types, instead of using values.

Example: drop the first element of a tuple.

In [ ]:
tuple_tail1(t) = t[2:end]

In [ ]:
tuple_tail1((1,2,"hi"))

In [ ]:
@code_typed tuple_tail1((1,2,"hi"))

Not good. Key information is represented as integers, and when the compiler sees an integer it generally assumes it doesn't know its value.

- The compiler counts 1, infinity
- The compiler can match things but cannot do arithmetic or comparisons
- It's very good at knowing the types of function arguments

In [ ]:
argtail(a, rest...) = rest
tupletail(t) = argtail(t...)

In [ ]:
tupletail((1,2,"hi"))

In [ ]:
@code_typed tupletail((1,2,"hi"))

### Exercise

Write a type-inferable function to...

1. reverse a tuple
1. take every other element of a tuple
2. interleave the elements of two tuples

### Real-ish example: computing the shape of an indexing operation

In [ ]:
index_shape(a::Array, idxs) = ish(a, 1, idxs...)

ish(a, i,   ::Real...)         = ()
ish(a, i,   ::Colon,  rest...) = (size(a,i), ish(a,i+1,rest...)...)
ish(a, i, iv::Vector, rest...) = (length(iv), ish(a,i+1,rest...)...)
ish(a, i,   ::Real,   rest...) = ish(a,i+1,rest...)

In [ ]:
index_shape(rand(3,4,5), (1,:,[1,2]))

In [ ]:
index_shape(rand(3,4,5), (:,2,[1,2,1,2,1,2,1,2]))

## Trick 2: look up "trait" values and re-dispatch

### Functions of types

In [ ]:
widen(::Type{Float32}) = Float64

In [ ]:
widen(Float32)

In [ ]:
# We use this for type promotion
promote_type(Int64, Float64)

This can be used to compute attributes of types, then dispatch on those values.

In [ ]:
# Sample trait
abstract IteratorSize
immutable SizeUnknown <: IteratorSize end
immutable HasLength <: IteratorSize end
immutable HasShape <: IteratorSize end
immutable IsInfinite <: IteratorSize end

Now we can define a method that says which value of the trait a certain type has.

This is like using dispatch as a lookup table to find out properties of a combination of values.

In [ ]:
iteratorsize{T<:AbstractArray}(::Type{T}) = HasShape()

iteratorsize{I1,I2}(::Type{Zip2{I1,I2}}) = zip_iteratorsize(iteratorsize(I1),iteratorsize(I2))

zip_iteratorsize(a, b) = SizeUnknown()
zip_iteratorsize{T}(isz::T, ::T) = isz
zip_iteratorsize(::HasLength, ::HasShape) = HasLength()
zip_iteratorsize(::HasShape, ::HasLength) = HasLength()

In [ ]:
# `collect` gives you all the elements from an iterator as an array
collec(itr) = _collec(itr, eltype(itr), Base.iteratorsize(itr))

In [ ]:
function _collec(itr, T, ::Base.HasLength)
    a = Array{T,1}(length(itr))
    i = 0
    for x in itr
        a[i+=1] = x
    end
    return a
end

In [ ]:
function _collec(itr, T, ::Base.SizeUnknown)
    a = Array{T,1}(0)
    for x in itr
        push!(a, x)
    end
    return a
end